In [0]:
# clustering hackers via hacker metadata
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('kmeans_consulting_proj').getOrCreate()
data = spark.read.csv("/FileStore/shared_uploads/sejal@ibm.com/hack_data.csv", inferSchema=True, header=True)

In [0]:
data.head()

Out[12]: Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [0]:
feat_cols = [col for col in data.columns if col != 'Location']

In [0]:
assembler = VectorAssembler(inputCols=feat_cols, outputCol='features')
final_data = assembler.transform(data)
final_data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')
scaler_model = scaler.fit(final_data)
cluster_final_data = scaler_model.transform(final_data)

In [0]:
cluster_final_data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)
-- scaledFeatures: vector (nullable = true)

In [0]:
# need 2 kmeans models, one for k = 2 and one for k = 3
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans3 = KMeans(featuresCol='scaledFeatures', k=3)

In [0]:
model_k2 = kmeans2.fit(cluster_final_data)
model_k3 = kmeans3.fit(cluster_final_data)

In [0]:
model_k3.transform(cluster_final_data).select('prediction').groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+

In [0]:
model_k2.transform(cluster_final_data).select('prediction').groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

In [0]:
# goes to show that k = 2 was the most optimal for this task given that the hackers did an equal number of attacks